In [1]:
import pandas as pd
import warnings
from dateutil.parser import parse
warnings.filterwarnings('ignore')
import os.path

### Load the buildings and cpq data

In [2]:
PATH = '../data/'
PROCESSED_PATH = '../processedData/'

cpq = pd.read_csv(PATH + 'ZayoHackathonData_CPQs.csv')
buildings = pd.read_csv(PATH + 'ZayoHackathonData_Buildings.csv')
opportunities = pd.read_csv(PATH + 'ZayoHackathonData_Opportunities.csv')
accounts = pd.read_csv(PATH + 'ZayoHackathonData_Accounts.csv')
sites = pd.read_csv(PATH + 'ZayoHackathonData_Sites.csv')

### drop rows that have duplicates on 'Account ID', 'Product Group', 'Building ID'

In [ ]:
cpq['CreatedDate'] = cpq['CreatedDate'].apply(lambda x : parse(x))
cpq.sort_values(by='CreatedDate', inplace=True)
cpq.drop_duplicates(['Account ID', 'Product Group', 'Building ID'], inplace=True)
opportunities.drop_duplicates(['Account ID', 'Product Group', 'Building ID'], inplace=True)

### Get the count of records by the attribute 'On Zayo Network Status'

In [ ]:
cpq['On Zayo Network Status'].value_counts()

In [ ]:
cpq.columns

### Merge the cpq and buildings by doing an inner join

In [ ]:
cpq_business = pd.merge(cpq, buildings, how='inner')

In [ ]:
cpq_business.head()

In [ ]:
# len(cpq.loc[cpq['Building ID'] == 'Bldg-108671']), len(cpq_business.loc[cpq_business['Building ID'] == 'Bldg-108671'])

### converting the building ID to string values

In [ ]:
cpq_business['Building ID'] = cpq_business['Building ID'].apply(str)
cpq_business['Building ID'].unique()

### Select only those records that are not on Zayo Network

In [ ]:
cpq_status = cpq_business.loc[cpq_business['On Zayo Network Status'] == 'Not on Zayo Network']

In [ ]:
cpq_status.rename(columns={' X36 MRC List ': 'X36 MRC', ' X36 NRR List ': 'X36 NRR', ' X36 NPV List ': 'X36 NPV'}, inplace=True)

### Get price in numbers

In [ ]:
cpq_status['X36 MRC'] = cpq_status['X36 MRC'].replace('[\$,)]','',regex=True).astype(float)
cpq_status['X36 NRR'] = cpq_status['X36 NRR'].str.replace(r'[$,]', '')
cpq_status['X36 NRR'] = cpq_status['X36 NRR'].str.replace('-', '0')
cpq_status['X36 NPV'] = cpq_status['X36 NPV'].str.replace(r'[$,]', '').replace('-', '0')
cpq_status['X36 NPV'] = cpq_status['X36 NPV'].str.replace('-', '0')
cpq_status[['X36 NRR','X36 NPV']] = cpq_status[['X36 NRR','X36 NPV']].apply(pd.to_numeric)
cpq_status[' Estimated Build Cost '] = cpq_status[' Estimated Build Cost '].replace('[\$,)]','',regex=True).astype(float)

cpq_status.head()

### find number of distinct buildings on Zayo network

In [ ]:
len(cpq_status['Building ID'].unique())

### Filter out the records based on the state

In [ ]:
cpq_CO = cpq_status.loc[cpq_status['State'] == 'CO']
cpq_TX = cpq_status.loc[cpq_status['State'] == 'TX']
cpq_GA = cpq_status.loc[cpq_status['State'] == 'GA']

### For each individual state, calculate the profit incurred by each building

In [ ]:
CO_profit = cpq_CO.groupby(cpq_CO['Building ID'])['X36 NPV'].sum().reset_index()
CO_profit.sort_values(by='Building ID', inplace=True)
TX_profit = cpq_TX.groupby(cpq_TX['Building ID'])['X36 NPV'].sum().reset_index()
TX_profit.sort_values(by='Building ID', inplace=True)
GA_profit = cpq_GA.groupby(cpq_GA['Building ID'])['X36 NPV'].sum().reset_index()
GA_profit.sort_values(by='Building ID', inplace=True)

### For each building, get the total number of accounts associated with the building

In [ ]:
CO_accounts = cpq_CO.groupby(cpq_CO['Building ID'])['Account ID'].count().reset_index()
CO_accounts.sort_values(by='Building ID', inplace=True)
TX_accounts = cpq_TX.groupby(cpq_TX['Building ID'])['Account ID'].count().reset_index()
TX_accounts.sort_values(by='Building ID', inplace=True)
GA_accounts = cpq_GA.groupby(cpq_GA['Building ID'])['Account ID'].count().reset_index()
GA_accounts.sort_values(by='Building ID', inplace=True)

In [ ]:
CO_accounts.head()

### Get the estimated build cost for each state

In [ ]:
build_cost_CO = cpq_CO.groupby(['Building ID',' Estimated Build Cost ']).size().reset_index().rename(columns={0:'count'})
build_cost_TX = cpq_TX.groupby(['Building ID',' Estimated Build Cost ']).size().reset_index().rename(columns={0:'count'})
build_cost_GA = cpq_GA.groupby(['Building ID',' Estimated Build Cost ']).size().reset_index().rename(columns={0:'count'})

### Total profit = Profit incurred by each building - Estimated build cost

In [ ]:
CO_profit['Estimated Build Cost'] = build_cost_CO[' Estimated Build Cost ']
CO_profit['Profit Including Build Cost'] = CO_profit['X36 NPV'] - CO_profit['Estimated Build Cost']
CO_profit['Number of Accounts'] = CO_accounts['Account ID']

In [ ]:
TX_profit['Estimated Build Cost'] = build_cost_TX[' Estimated Build Cost ']
TX_profit['Profit Including Build Cost'] = TX_profit['X36 NPV'] - TX_profit['Estimated Build Cost']
TX_profit['Number of Accounts'] = TX_accounts['Account ID']

In [ ]:
GA_profit['Estimated Build Cost'] = build_cost_GA[' Estimated Build Cost ']
GA_profit['Profit Including Build Cost'] = GA_profit['X36 NPV'] - GA_profit['Estimated Build Cost']
GA_profit['Number of Accounts'] = GA_accounts['Account ID']

In [ ]:
CO_profit.sort_values(by='Profit Including Build Cost', ascending=False, inplace=True)
TX_profit.sort_values(by='Profit Including Build Cost', ascending=False, inplace=True)
GA_profit.sort_values(by='Profit Including Build Cost', ascending=False, inplace=True)

In [ ]:
CO_profit.head()

In [ ]:
TX_profit.head()

In [ ]:
GA_profit.head()

### Merge the profits dataframe with the original datafrae

In [ ]:
co_buildings_latlong = pd.merge(CO_profit, cpq_status, on='Building ID', how='inner')
tx_buildings_latlong = pd.merge(TX_profit, cpq_status, on='Building ID', how='inner')
ga_buildings_latlong = pd.merge(GA_profit, cpq_status, on='Building ID', how='inner')

In [ ]:
co_buildings_latlong.head()

### Generate the profits csv by combining the state dataframes. This csv would be used to generate the data table for the first visualization

In [ ]:
co_buildings_latlong = co_buildings_latlong[['Building ID', 'X36 NPV_x', 'Estimated Build Cost', 'Profit Including Build Cost',
                                            'Latitude', 'Longitude', 'State', 'Number of Accounts', 'Street Address',
                                            'Postal Code', 'Net Classification', 'Type']]
co_buildings_latlong.drop_duplicates(['Building ID'], inplace=True)
tx_buildings_latlong = tx_buildings_latlong[['Building ID', 'X36 NPV_x', 'Estimated Build Cost', 'Profit Including Build Cost',
                                            'Latitude', 'Longitude', 'State', 'Number of Accounts', 'Street Address',
                                            'Postal Code', 'Net Classification', 'Type']]
tx_buildings_latlong.drop_duplicates(['Building ID'], inplace=True)
ga_buildings_latlong = ga_buildings_latlong[['Building ID', 'X36 NPV_x', 'Estimated Build Cost', 'Profit Including Build Cost',
                                            'Latitude', 'Longitude', 'State', 'Number of Accounts', 'Street Address',
                                            'Postal Code', 'Net Classification', 'Type']]
ga_buildings_latlong.drop_duplicates(['Building ID'], inplace=True)

In [ ]:
profits = pd.concat([co_buildings_latlong, tx_buildings_latlong, ga_buildings_latlong])

In [ ]:
profits.to_csv('profits.csv')

### Identify the total number of buildings for each state and write that to a csv

In [ ]:
co_sum = CO_profit['Profit Including Build Cost'].sum()
buildings_co = len(CO_profit)

In [ ]:
tx_sum = TX_profit['Profit Including Build Cost'].sum()
buildings_tx = len(TX_profit)

In [ ]:
ga_sum = GA_profit['Profit Including Build Cost'].sum()
buildings_ga = len(GA_profit)

In [ ]:
import csv
vals = [['CO', co_sum, buildings_co], ['TX', tx_sum, buildings_tx], ['GA', ga_sum, buildings_ga]]

with open('profit_by_state.csv','wb') as f:
    w = csv.writer(f)
    w.writerow(['State','Total Profit', 'Number of Buildings'])
    for v in vals:
        w.writerow(v)

### getting a glimpse of the data in the opportunities file

In [ ]:
opportunities.head(3)

### isolating opportunities that haven't been 'Closed - Lost'

In [3]:
opportunities_not_lost = opportunities[opportunities['StageName'] != 'Closed - Lost'].groupby('Account ID')['Building ID'].count().reset_index()

### count the number of buildings on every account grouped by network status

In [4]:
count_bldg_opps = opportunities.groupby(['Account ID','On Zayo Network Status'])['Building ID'].count()
count_bldg_opps = count_bldg_opps.reset_index()

In [ ]:
count_bldg_opps.head()

### converting Total BRR from strings to floats

In [5]:
accounts[' Total BRR '] = accounts[' Total BRR '].map(lambda tbrr: float(tbrr.split('$')[1]
                                                                         .replace(',','')
                                                                         .replace('-','0.0')))

accounts[' Total BRR '] = accounts[' Total BRR '].astype(float)

In [ ]:
accounts.head()

### isolating accounts having Total BRR >= 500,00 (high-revenue accounts) and On Zayo Network

In [6]:
high_rev_acc = accounts[accounts[' Total BRR '] >= 500000]
high_rev_acc_opps = pd.merge(high_rev_acc, count_bldg_opps, on=['Account ID'], how='inner')
high_rev_acc_opps_net = pd.DataFrame(high_rev_acc_opps[high_rev_acc_opps['On Zayo Network Status'] == 'On Zayo Network'])

In [7]:
high_rev_acc_opps_net.drop([' DandB Revenue ', 'DandB Total Employees'], axis=1, inplace=True)

In [ ]:
high_rev_acc_opps_net.columns

In [8]:
high_rev_acc_opps_net.rename(columns={'Building ID': 'Total Buildings'}, inplace=True)

### sorting by number of on-net buildings

In [9]:
high_rev_acc_opps_net.sort_values(by='Total Buildings', ascending=False, inplace=True)

In [10]:
high_rev_acc_opps_net.head()

,Account ID,Industry,Vertical,Total BRR,AnnualRevenue,NumberOfEmployees,On Zayo Network Status,Total Buildings
15,Acct-000002,Telecommunications,National Carrier/ISP,3344939.0,"$25,378,360,491.80",43350,On Zayo Network,133
5,Acct-000013,Telecommunications,National Carrier/ISP,5117529.0,"$10,779,768,901.96",40000,On Zayo Network,112
20,Acct-000009,Telecommunications,National Carrier/ISP,1915889.0,"$5,374,799,000.00",12626,On Zayo Network,109
17,Acct-000053,Telecommunications,National Carrier/ISP,2950069.0,"$16,503,800,000.00",43000,On Zayo Network,70
26,Acct-000072,Telecommunications,National Carrier/ISP,1307326.0,"$109,048,721.12",431,On Zayo Network,61


In [ ]:
high_rev_acc_opps_net.shape

### saving to csv for use in visualization

In [11]:
saveToFile = os.path.join(PROCESSED_PATH, 'Opportunities_with_Current_High_Revenue_Accounts_On_Net.csv')
high_rev_acc_opps_net.to_csv(saveToFile, index = False)

### isolating accounts with no BRR (potential accounts)

In [12]:
zero_rev_acc = accounts[accounts[' Total BRR '] == 0]
zero_rev_acc_opps = pd.merge(zero_rev_acc, opportunities_not_lost,\
                                    on=['Account ID'],\
                                    how='inner').sort_values(by='Building ID', ascending=False)

In [ ]:
zero_rev_acc_opps.columns

In [13]:
zero_rev_acc_opps.drop(labels=[' Total BRR ', ' AnnualRevenue ',
       'NumberOfEmployees', ' DandB Revenue ', 'DandB Total Employees'], axis=1, inplace=True)

In [14]:
zero_rev_acc_opps.rename(columns={'Building ID': 'Total Buildings'}, inplace=True)

### sorting by number of on-net buildings

In [15]:
zero_rev_acc_opps.sort_values(by='Total Buildings', ascending=False, inplace=True)

In [ ]:
zero_rev_acc_opps.head()

### saving to csv for use in visualization

In [ ]:
saveToFile = os.path.join(PROCESSED_PATH, 'Opportunities_with_Potential_Accounts.csv')
zero_rev_acc_opps.to_csv(saveToFile, index = False)

### isolating on-net sites

In [16]:
sites_on_net = sites[sites['On Zayo Network Status'] != 'Not on Zayo Network'].groupby(['Account ID'])['Building ID'].count().reset_index().sort_values(by='Building ID', ascending=False)

### isolating sites not on net

In [17]:
sites_no_net = sites[sites['On Zayo Network Status'] == 'Not on Zayo Network'].groupby(['Account ID'])['Building ID'].count().reset_index().sort_values(by='Building ID', ascending=False)

### renaming columns to something more meaningful

In [18]:
sites_on_net.rename(columns={'Building ID': '# Buildings on Net'}, inplace=True)
sites_no_net.rename(columns={'Building ID': '# Buildings not on Net'}, inplace=True)

### getting the total number of on-net and off-net buildings associated with every account

In [19]:
potential_accounts_buildings_info = (pd.merge(sites_on_net, sites_no_net,\
         on=['Account ID'],\
         how='outer').sort_values(by='# Buildings on Net', ascending=False)).fillna(0)

potential_accounts_buildings_info['# Buildings not on Net'] = potential_accounts_buildings_info['# Buildings not on Net'].astype(int)

### extending accounts.csv to have 2 more columns: number of buildings on-net and number off-net

In [20]:
potential_accounts_buildings_info_tbrr = pd.merge(accounts, potential_accounts_buildings_info,\
         on=['Account ID'],\
         how='inner')

### isolating only those accounts with Total BRR > 0 and dropping unwanted columns

In [21]:
potential_accounts_buildings_info_tbrr = pd.DataFrame(potential_accounts_buildings_info_tbrr\
                                                      [potential_accounts_buildings_info_tbrr[' Total BRR '] > 0])

potential_accounts_buildings_info_tbrr.drop([' AnnualRevenue ','NumberOfEmployees',\
                                             ' DandB Revenue ', 'DandB Total Employees'], axis=1, inplace=True)

In [22]:
potential_accounts_buildings_info_tbrr.head()

,Account ID,Industry,Vertical,Total BRR,# Buildings on Net,# Buildings not on Net
0,Acct-000273,Telecommunications,Wireless,6084906.0,970.0,45
1,Acct-000013,Telecommunications,National Carrier/ISP,5117529.0,188.0,17
2,Acct-000036,Telecommunications,Wireless,5105015.0,75.0,587
3,Acct-000025,Telecommunications,National Carrier/ISP,4234871.0,228.0,32
4,Acct-000004,Telecommunications,Wireless,3510947.0,446.0,213


### sorting by number of on-net buildings

In [24]:
potential_accounts_buildings_info_tbrr_temp = potential_accounts_buildings_info_tbrr[potential_accounts_buildings_info_tbrr['# Buildings not on Net'] > 0] 
potential_accounts_buildings_info_tbrr_temp.sort_values(by='# Buildings on Net', ascending=False, inplace=True)

### saving to csv for use in visualization

In [28]:
saveToFile = os.path.join(PROCESSED_PATH, 'Untapped_Buildings_on_Current_Accounts_On_Net.csv')
potential_accounts_buildings_info_tbrr_temp.to_csv(saveToFile, index = False)